In [72]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras import metrics
from keras.layers import Conv1D, GlobalMaxPooling1D, Activation


import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [43]:
df=pd.read_csv('SPAM text message 20170820 - Data.csv',encoding='latin-1')

In [44]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
Category    5572 non-null object
Message     5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [46]:
# df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [47]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [50]:
df['Message']=df.Message.str.strip()
df['Category']=df.Category.str.strip()

In [51]:
df['Message']=df.Message.str.lower()

In [52]:
df.Category.value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [53]:
df.head()

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [80]:
num_max = 1500
le = LabelEncoder()
tags = le.fit_transform(df.Category)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(df.Message)
mat_texts = tok.texts_to_matrix(df.Message,mode='count')

In [81]:
len(tags)

5572

In [82]:
tags

array([0, 0, 1, ..., 0, 0, 0])

In [83]:
mat_texts

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [84]:
mat_texts.shape

(5572, 1500)

In [88]:
X_train, X_test, Y_train, Y_test = train_test_split(mat_texts,tags, test_size=0.25, random_state=42)

In [89]:
def get_simple_model():
    model = Sequential()
    model.add(Embedding(1000,20,input_length=num_max))
    model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])
    return model
    
def check_model(model,x,y):
    model.fit(x,y,batch_size=32,epochs=3,verbose=1,validation_split=0.2)

m = get_simple_model()
check_model(m,X_train,Y_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1500, 20)          20000     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1498, 64)          3904      
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 64)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 256)               16640     
_________________________________________________________________
dropout_22 (Dropout)         (None, 256)               0         
_________________________________________________________________
activation_11 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 257       
__________

In [90]:

pred = m.predict_classes(X_test)
acc = m.evaluate(X_test, Y_test)
# proba_ltsm = m.predict_proba(X_test)
from sklearn.metrics import confusion_matrix,classification_report
print("Test loss is {0:.2f} accuracy is {1:.2f}  ".format(acc[0],acc[1]))
print(confusion_matrix(pred, Y_test))
print(classification_report(pred, Y_test))

1393/1393 [==============================] - 1s 595us/step
Test loss is 0.37 accuracy is 0.87  
[[1207  186]
 [   0    0]]
              precision    recall  f1-score   support

           0       1.00      0.87      0.93      1393
           1       0.00      0.00      0.00         0

   micro avg       0.87      0.87      0.87      1393
   macro avg       0.50      0.43      0.46      1393
weighted avg       1.00      0.87      0.93      1393

